In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

Import packages ˆ

In [2]:
#load datasets:

df_cycledata = pd.read_csv("data/cycledata_utf8.csv", delimiter=";", encoding= "utf-8")

df_weatherdata = pd.read_csv('data/weather.csv', on_bad_lines = 'warn')

df_cycledata.head(5)


Skipping line 36628: expected 6 fields, saw 10
Skipping line 36664: expected 6 fields, saw 10



,Trafikkregistreringspunkt,Navn,Vegreferanse,Fra,Til,Dato,Fra tidspunkt,Til tidspunkt,Felt,Trafikkmengde,...,Ikke gyldig lengde,Lengdekvalitetsgrad (%),"< 5,6m",">= 5,6m","5,6m - 7,6m","7,6m - 12,5m","12,5m - 16,0m",">= 16,0m","16,0m - 24,0m",">= 24,0m"
0,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2022-08-31T00:00+02:00,2022-08-31T01:00+02:00,2022-08-31,00:00,01:00,1,1,...,1,"0,00",-,-,-,-,-,-,-,-
1,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2022-08-31T00:00+02:00,2022-08-31T01:00+02:00,2022-08-31,00:00,01:00,2,5,...,5,"0,00",-,-,-,-,-,-,-,-
2,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2022-08-31T00:00+02:00,2022-08-31T01:00+02:00,2022-08-31,00:00,01:00,Totalt i retning Danmarksplass,5,...,5,"0,00",-,-,-,-,-,-,-,-
3,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2022-08-31T00:00+02:00,2022-08-31T01:00+02:00,2022-08-31,00:00,01:00,Totalt i retning Florida,1,...,1,"0,00",-,-,-,-,-,-,-,-
4,17510B2483952,Gamle Nygårdsbru sykkel,KV256 S2D1 m75,2022-08-31T00:00+02:00,2022-08-31T01:00+02:00,2022-08-31,00:00,01:00,Totalt,6,...,6,"0,00",-,-,-,-,-,-,-,-


In [3]:
df_weatherdata.head(5)

,Dato,Tid,Solskinstid,Lufttemperatur,Relativ luftfuktighet,Vindstyrke
0,2022-08-31,00:00,0.0,13.7,88.0,0.0
1,2022-08-31,00:10,0.0,13.9,87.0,0.0
2,2022-08-31,00:20,0.0,14.1,87.0,0.0
3,2022-08-31,00:30,0.0,14.1,87.0,0.0
4,2022-08-31,00:40,0.0,14.3,86.0,0.0


In [4]:
#filter lines i dont give a shit about and connect date and time

filtered_df_cycledata = df_cycledata[['Dato','Fra tidspunkt','Felt', 'Trafikkmengde']]
filtered_df_cycledata = filtered_df_cycledata[filtered_df_cycledata['Felt'] == 'Totalt']
filtered_df_cycledata = filtered_df_cycledata.drop('Felt', axis=1)
filtered_df_cycledata['Datetime'] = pd.to_datetime(filtered_df_cycledata['Dato'] + ' ' + filtered_df_cycledata['Fra tidspunkt'])
filtered_df_cycledata = filtered_df_cycledata.drop(['Dato', 'Fra tidspunkt'], axis= 1)
filtered_df_cycledata.head(5)


,Trafikkmengde,Datetime
4,6,2022-08-31 00:00:00
9,0,2022-08-31 01:00:00
14,2,2022-08-31 02:00:00
19,0,2022-08-31 03:00:00
24,0,2022-08-31 04:00:00


In [5]:
# the same for weather data

filtered_df_weatherdata = df_weatherdata.drop('Vindstyrke', axis=1)
filtered_df_weatherdata['Datetime'] = pd.to_datetime(filtered_df_weatherdata['Dato'] + ' ' + filtered_df_weatherdata['Tid'])
filtered_df_weatherdata = filtered_df_weatherdata.drop(['Dato', 'Tid'], axis= 1)
filtered_df_weatherdata['Datetime'] = filtered_df_weatherdata['Datetime'].dt.floor('H')
filtered_df_weatherdata['Solskinstid'].replace(9999.99, np.nan, inplace=True)
filtered_df_weatherdata.fillna(0, inplace=True)
filtered_df_weatherdata['Lufttemperatur'].replace(9999.99, np.nan, inplace=True)
filtered_df_weatherdata['Relativ luftfuktighet'].replace(9999.99, np.nan, inplace=True)
filtered_df_weatherdata.fillna(method='ffill', inplace=True)
filtered_df_weatherdata.head(5)

,Solskinstid,Lufttemperatur,Relativ luftfuktighet,Datetime
0,0.0,13.7,88.0,2022-08-31
1,0.0,13.9,87.0,2022-08-31
2,0.0,14.1,87.0,2022-08-31
3,0.0,14.1,87.0,2022-08-31
4,0.0,14.3,86.0,2022-08-31


In [6]:
#Merge the data into one file

merged_df = pd.merge(filtered_df_cycledata, 
                     filtered_df_weatherdata[['Solskinstid','Lufttemperatur','Relativ luftfuktighet','Datetime']], 
                     on= 'Datetime',
                     how='left')

merged_df = merged_df.groupby('Datetime').mean().reset_index()

merged_df.head(20)

,Datetime,Trafikkmengde,Solskinstid,Lufttemperatur,Relativ luftfuktighet
0,2022-08-31 00:00:00,6.0,0.000000,14.066667,86.833333
1,2022-08-31 01:00:00,0.0,0.000000,14.450000,87.000000
2,2022-08-31 02:00:00,2.0,0.000000,14.766667,87.000000
3,2022-08-31 03:00:00,0.0,0.000000,14.966667,85.166667
4,2022-08-31 04:00:00,0.0,0.000000,14.816667,86.166667
5,2022-08-31 05:00:00,9.0,0.000000,14.983333,84.000000
6,2022-08-31 06:00:00,67.0,0.000000,15.233333,81.500000
7,2022-08-31 07:00:00,306.0,3.516667,15.666667,78.833333
8,2022-08-31 08:00:00,365.0,6.066667,16.116667,75.000000
9,2022-08-31 09:00:00,153.0,2.650000,16.733333,69.666667


In [7]:
# save the merged_df as csv to eventual later use
merged_df.to_csv('data/cycleandweather.csv')

# Data Visualization
Correspondence between Relativ Luftfuktighet og solskinnstid?
checking the correspondence for this summer, may until august

In [8]:

summer_df = merged_df[(merged_df['Datetime'] >= '2023-05-01') & (merged_df['Datetime'] <= '2023-08-05')]

#scale solskinstid so i can see more differance
solskinstid_scaled = summer_df['Solskinstid'] * 10


fig = px.line(summer_df, x= 'Datetime', y = ['Relativ luftfuktighet'],
              title='Relativ fuktighet (linje) Solskinstid (scatter)', 
              color_discrete_sequence=['blue'])

scatter_trace = px.scatter(summer_df, x='Datetime', y= solskinstid_scaled,
                           color_discrete_sequence=['red'] )

fig.add_trace(scatter_trace.data[0])
fig.show()

very nice, I assumed low relativ fuktighet was supposed to be low when there is a lot of sunshine.<br>
The correspondence is very clear, as clear as a sunny day 😂
<br><br>
onto my next task, I want to see how the trafikkmengde changes based on a yearly timeframe<br>
my assumption is lower in the winter, higher in the summer



In [9]:
# check which idot data that misses the whole ass summer

figcycle = px.line(filtered_df_cycledata, x= 'Datetime', y='Trafikkmengde', title= 'Trafikkmengde over et år')
figcycle


## Correspondence temperature and cyclists 
<br>
I have a theory that when it is cold, there are less cyclists. <br>let's check it out:

In [10]:
#will create a data frame of values from a colder month, so November

november_data = merged_df[(merged_df['Datetime'] >= '2022-11-01') & (merged_df['Datetime'] <= '2022-11-30')]

# I will scale temperature by the square to the amount
scaled_temp_november = november_data['Lufttemperatur'] ** 2

coldfig = px.line(november_data, x= 'Datetime', y= ['Trafikkmengde', scaled_temp_november], title= 'Antall syklister med Temperatur sammenligning')
coldfig

Det viste seg at det var veldig lite sammenheng mellom temperatur og antall syklister som sykler over bruen. <br><br>
sjekke om det er noe sammenheng mellom regn og antall syklister:

In [11]:

scaled_relativ_fuktighet = november_data['Relativ luftfuktighet'] * 1.8

regnfig = px.line(november_data, x= 'Datetime', y= ['Trafikkmengde', scaled_relativ_fuktighet], title='Luftfuktighet(regnsannsynlighet) vs Syklister')
regnfig


Veldig lite sammenheng mellom begge disse... <br>
ikke sånn som jeg forventet det.<br>
hva er det beste været man kan ha? det må jo være Relativ (temperatur + solskinnstid) * 2 - relativ luftfuktighet. <br>
sjekke om det har noen sammenheng..

In [12]:


fukt_temp_sol = (november_data['Lufttemperatur'] + november_data['Solskinstid']) * 2 - november_data['Relativ luftfuktighet']

wheaterfig = px.line(november_data, x= 'Datetime', y= ['Trafikkmengde', fukt_temp_sol], title= 'vær vs syklister')
wheaterfig


## Konklusjon: 
<br>Virker som det er veldig lite til ingen sammenheng mellom vær og antall syklister som krysser bruen.

# forskjeller vår / sommer / høst / vinter

la oss se hva forskjellen fra antall syklister vår / sommer / høst / vinter

In [13]:

# Define the date ranges for each season
spring_start = '2023-03-01'
spring_end = '2023-05-31'
summer_start = '2023-06-01'
summer_end = '2023-08-28'
autumn_start = '2022-09-01'
autumn_end = '2022-11-30'
winter_start = '2022-12-01'
winter_end = '2023-02-28'

# Create DataFrames for each season
spring_data = merged_df[(merged_df['Datetime'] >= spring_start) & (merged_df['Datetime'] <= spring_end)]
summer_data = merged_df[(merged_df['Datetime'] >= summer_start) & (merged_df['Datetime'] <= summer_end)]
autumn_data = merged_df[(merged_df['Datetime'] >= autumn_start) & (merged_df['Datetime'] <= autumn_end)]
winter_data = merged_df[(merged_df['Datetime'] >= winter_start) & (merged_df['Datetime'] <= winter_end)]

# Calculate the number of hours in each season
num_hours_spring = len(spring_data)
num_hours_summer = len(summer_data)
num_hours_autumn = len(autumn_data)
num_hours_winter = len(winter_data)

# Create DataFrames with index from 0 to the number of hours in each season
spring_data['Index'] = range(num_hours_spring)
summer_data['Index'] = range(num_hours_summer)
autumn_data['Index'] = range(num_hours_autumn)
winter_data['Index'] = range(num_hours_winter)

# Select the desired number of rows to make the lengths consistent
num_rows_to_select = min(num_hours_spring, num_hours_summer, num_hours_autumn, num_hours_winter)
spring_data = spring_data.iloc[:num_rows_to_select]
summer_data = summer_data.iloc[:num_rows_to_select]
autumn_data = autumn_data.iloc[:num_rows_to_select]
winter_data = winter_data.iloc[:num_rows_to_select]

# Create an index DataFrame
indeces = pd.DataFrame({'Index': [i for i in range(num_rows_to_select)]})

# Merge the DataFrames
season_df = pd.merge(indeces, spring_data[['Index', 'Trafikkmengde']], on='Index', how='left')

season_df.rename(columns={'Trafikkmengde' : 'Spring'}, inplace=True)

print(season_df.head(10))

season_df = pd.merge(season_df, summer_data[['Index', 'Trafikkmengde']], on='Index', how='left')

season_df.rename(columns={'Trafikkmengde' : 'Summer'}, inplace=True)

season_df = pd.merge(season_df, autumn_data[['Index', 'Trafikkmengde']], on='Index', how='left')

season_df.rename(columns={'Trafikkmengde' : 'Autumn'}, inplace=True)

season_df = pd.merge(season_df, winter_data[['Index', 'Trafikkmengde']], on='Index', how='left')

season_df.rename(columns={'Trafikkmengde' : 'Winter'}, inplace=True)




# Create the line plot
season_fig = px.line(season_df, x='Index', y=['Spring', 'Summer', 'Autumn', 'Winter'])
season_fig.update_xaxes(title_text='Days in Season')
season_fig.update_yaxes(title_text='Trafikkmengde')

# Show the figure
season_fig.show()





   Index  Spring
0      0     3.0
1      1     1.0
2      2     1.0
3      3     0.0
4      4     1.0
5      5     3.0
6      6    28.0
7      7   108.0
8      8   159.0
9      9    59.0


/var/folders/rm/fqwwr80d5fz6d4gnycfgb0dh0000gn/T/ipykernel_72824/3342743792.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/rm/fqwwr80d5fz6d4gnycfgb0dh0000gn/T/ipykernel_72824/3342743792.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/rm/fqwwr80d5fz6d4gnycfgb0dh0000gn/T/ipykernel_72824/3342743792.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

ser ut som det er flest syklister på høsten, i året 2022/2023 <br>
(litt sjokkerende)
<br>
la oss sjekke gjennomsnittet:

In [14]:
print('Vår:', round(spring_data['Trafikkmengde'].mean(), 2), 'per Time')
print('Sommer:', round(summer_data['Trafikkmengde'].mean(), 2), 'per Time')
print('Høst:', round(autumn_data['Trafikkmengde'].mean(), 2), 'per Time')
print('Vinter:', round(winter_data['Trafikkmengde'].mean(), 2), 'per Time')

Vår: 39.71 per Time
Sommer: 43.1 per Time
Høst: 59.53 per Time
Vinter: 26.81 per Time


Ser ut som den visuelle dataen stemmer fint med gjennomsnittet.

fikk lyst til å sjekke værdata ifht syklist data en gang til.

In [15]:
#which hours of the day is the most cyclists??
date_to_hour_data = merged_df[(merged_df['Datetime'] >= '2022-09-01') & (merged_df['Datetime'] <= '2023-08-25')]
hours = range(len(date_to_hour_data))
hours2 = [i%24 for i in hours]
date_to_hour_data['Hours'] = hours2

hist = px.histogram(date_to_hour_data, x= 'Hours', y= 'Trafikkmengde')
hist


/var/folders/rm/fqwwr80d5fz6d4gnycfgb0dh0000gn/T/ipykernel_72824/1260919075.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Som forventet er det flest mennesker på vei til jobb 06-08 og på vei hjem 14 - 16

Lage og teste ut en liten modell.

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

In [17]:






Xfirst = merged_df.copy()

Xfirst['Month'] = Xfirst['Datetime'].dt.month
Xfirst['Day'] = Xfirst['Datetime'].dt.day
Xfirst['Hour'] = Xfirst['Datetime'].dt.hour

Xfirst.dropna(axis= 0, inplace=True)

X = Xfirst.drop('Trafikkmengde', axis=1)

y = Xfirst['Trafikkmengde']

X_nodate = X.drop('Datetime', axis= 1)

print(X_nodate.tail(5))

train_x, test_x, train_y, test_y = train_test_split(X_nodate, y, test_size=0.2, random_state=42)


reg = RandomForestRegressor(max_depth=4, min_samples_split=5, n_estimators=500,
                      oob_score=True, random_state=42, warm_start=True)

reg.fit(train_x, train_y)

print(X_nodate)



      Solskinstid  Lufttemperatur  Relativ luftfuktighet  Month  Day  Hour
8754          0.0       15.316667              73.166667      8   30    19
8755          0.0       14.800000              73.500000      8   30    20
8756          0.0       14.000000              76.333333      8   30    21
8757          0.0       13.583333              80.500000      8   30    22
8758          0.0       13.483333              82.333333      8   30    23
      Solskinstid  Lufttemperatur  Relativ luftfuktighet  Month  Day  Hour
0             0.0       14.066667              86.833333      8   31     0
1             0.0       14.450000              87.000000      8   31     1
2             0.0       14.766667              87.000000      8   31     2
3             0.0       14.966667              85.166667      8   31     3
4             0.0       14.816667              86.166667      8   31     4
...           ...             ...                    ...    ...  ...   ...
8754          0.0       1

In [18]:
print(test_x)

      Solskinstid  Lufttemperatur  Relativ luftfuktighet  Month  Day  Hour
3919          0.0        1.266667              59.500000      2   10     7
2786          0.0        4.100000              56.166667     12   25     2
3364          0.0        0.716667              85.666667      1   18     4
3064          0.0        1.283333              90.333333      1    5    16
6408          0.0        8.650000              73.833333      5   25     1
...           ...             ...                    ...    ...  ...   ...
4961          0.0        5.800000              61.833333      3   25    17
6453          0.0        8.800000              82.500000      5   26    22
4096          0.0        5.166667              77.000000      2   17    16
3541          0.0        7.100000              87.166667      1   25    13
1231          0.0        4.733333              70.333333     10   21     7

[1750 rows x 6 columns]


In [19]:


ypred = reg.predict(test_x)
#
mse = mean_squared_error(y_pred= ypred, y_true= test_y)

print(mse)

predvstest = px.histogram(test_x, x='Month', y= [test_y, ypred])

predvstest

reg.feature_importances_


2058.5669256838773


In [22]:

print(Xfirst.head(5))

newX = Xfirst.drop('Relativ luftfuktighet',  axis=1)
newX = newX.drop('Trafikkmengde', axis=1)
newX = newX.drop('Solskinstid', axis=1)
newX = newX.drop('Lufttemperatur', axis=1)

newX_noDate = newX.drop('Datetime', axis = 1)



train_x2, test_x2, train_y2, test_y2 = train_test_split(newX_noDate, y, test_size=0.2, random_state=42)

reg2 = RandomForestRegressor(max_depth=4, min_samples_split=5, n_estimators=500,
                      oob_score=True, random_state=42, warm_start=True)

reg2.fit(train_x2, train_y2)


             Datetime  Trafikkmengde  Solskinstid  Lufttemperatur  \
0 2022-08-31 00:00:00            6.0          0.0       14.066667   
1 2022-08-31 01:00:00            0.0          0.0       14.450000   
2 2022-08-31 02:00:00            2.0          0.0       14.766667   
3 2022-08-31 03:00:00            0.0          0.0       14.966667   
4 2022-08-31 04:00:00            0.0          0.0       14.816667   

   Relativ luftfuktighet  Month  Day  Hour  
0              86.833333      8   31     0  
1              87.000000      8   31     1  
2              87.000000      8   31     2  
3              85.166667      8   31     3  
4              86.166667      8   31     4  


RandomForestRegressor(max_depth=4, min_samples_split=5, n_estimators=500,
                      oob_score=True, random_state=42, warm_start=True)

In [24]:

ypred2 = reg2.predict(test_x2)

mse = mean_squared_error(test_y2, ypred2)

print(mse)

predvstest2 = px.histogram(test_x2, x='Month', y= [test_y2, ypred2])

predvstest2




2070.810828156633
